In [1]:
import scipy.io
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib 
import pandas as pd
import numpy as np
from numpy import random
import pickle
import timeit
from sklearn.model_selection import GridSearchCV

In [26]:
header = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('ml-latest-small/ratings_train.csv', names=header)
df = df[1:]
df['item_id'] = pd.to_numeric(df['item_id'])
df.head()

,user_id,item_id,rating,timestamp
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
5,1,70,3.0,964982400


In [27]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df,test_size=0.25)

train_data = pd.DataFrame(train_data)
test_data = pd.DataFrame(test_data)

In [29]:
n_u = len(df["user_id"].unique())
n_m = max(df["item_id"].unique())

R = np.zeros((n_u, n_m))
for line in train_data.itertuples():
    R[int(line[1])-1, int(line[2])-1] = line[3]  
    
T = np.zeros((n_u, n_m))
for line in test_data.itertuples():
    T[int(line[1])-1, int(line[2])-1] = line[3]

In [31]:
T

array([[0., 0., 4., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [3., 0., 0., ..., 0., 0., 0.],
       [5., 0., 0., ..., 0., 0., 0.]])

In [32]:

# Scoring Function: Root Mean Squared Error
def rmse_score(R, Q, P):
    I = R != 0  # Indicator function which is zero for missing data
    ME = I * (R - np.dot(P, Q.T))  # Errors between real and predicted ratings
    MSE = ME**2  
    return np.sqrt(np.sum(MSE)/np.sum(I))  # sum of squared errors

# Set parameters and initialize latent factors
f = 20  # Number of latent factor pairs
lmbda = 0.5 # Regularisation strength
gamma=0.01  # Learning rate
n_epochs = 50  # Number of loops through training data
P = 3 * np.random.rand(n_u, f) # Latent factors for users
Q = 3 * np.random.rand(n_m, f) # Latent factors for movies

# Stochastic GD
train_errors = []
test_errors = []
users,items = R.nonzero()      
for epoch in range(n_epochs):
    for u, i in zip(users,items):
        e = R[u, i] - np.dot(P[u, :], Q[i, :].T)  # Error for this observation
        P[u, :] += gamma * ( e * Q[i, :] - lmbda * P[u, :]) # Update this user's features
        Q[i, :] += gamma * ( e * P[u, :] - lmbda * Q[i, :])  # Update this movie's features
    train_errors.append(rmse_score(R,Q,P)) # Training RMSE for this pass
    test_errors.append(rmse_score(T,Q,P)) # Test RMSE for this pass

# Print how long it took
print("Run took %.2f seconds" % (timeit.default_timer() - start_time))
    
# Check performance by plotting train and test errors
fig, ax = plt.subplots()
ax.plot(train_errors, color="g", label='Training RMSE')
ax.plot(test_errors, color="r", label='Test RMSE')
snp.labs("Number of Epochs", "RMSE", "Error During Stochastic GD")
ax.legend()

NameError: name 'start_time' is not defined